In [42]:
%pip install pyriksdagen
from lxml import etree
import progressbar
from pyparlaclarin.read import paragraph_iterator, speeches_with_name,parlaclarin_to_txt,parlaclarin_to_md
from pyriksdagen.utils import protocol_iterators, download_corpus
from pyriksdagen.metadata import load_Corpus_metadata
import pyriksdagen
# We need a parser for reading in XML data
parser = etree.XMLParser(remove_blank_text=True)
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade pyparlaclarin

  Attempting uninstall: pyparlaclarin
    Found existing installation: pyparlaclarin 0.9.0
    Uninstalling pyparlaclarin-0.9.0:
      Successfully uninstalled pyparlaclarin-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show pyparlaclarin

Name: pyparlaclarin
Version: 0.9.1
Summary: Read, create, and modify Parla-Clarin XML files
Home-page: https://github.com/welfare-state-analytics/pyparlaclarin
Author: ninpnin
Author-email: vainoyrjanainen@icloud.com
License: MIT
Location: /home/laurinemeier/anaconda3/envs/swerick/lib/python3.12/site-packages
Requires: 
Required-by: pyriksdagen
Note: you may need to restart the kernel to use updated packages.


In [2]:
print(pyriksdagen.__spec__)
download_corpus(partitions=["politicians", "records"])

ModuleSpec(name='pyriksdagen', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7473401ad5d0>, origin='/home/laurinemeier/anaconda3/lib/python3.11/site-packages/pyriksdagen/__init__.py', submodule_search_locations=['/home/laurinemeier/anaconda3/lib/python3.11/site-packages/pyriksdagen'])


politicians.zip: 100%|██████████| 2.20M/2.20M [00:00<00:00, 28.9MiB/s]

11:11:34 [WARNING] (pyriksdagen): data already exists at the path 'data'. It will be overwritten once the download is finished.



records.zip: 100%|██████████| 1.60G/1.60G [00:20<00:00, 84.8MiB/s]


In [2]:
protocols = list(protocol_iterators(corpus_root="data/", start=1867, end=202122))

In [4]:
def oppna_data_to_dict(input_dict):
    """
    Load protocols with the new XML / HTML structure (from 2013 onwards)
    and convert it to a python dict with contents.
    """
    data = {}
    data["paragraphs"] = []

    # Metadata
    session = input_dict["dokumentstatus"]["dokument"]["rm"]
    session = session.replace("/", "")
    pid = input_dict["dokumentstatus"]["dokument"]["nummer"]
    date = input_dict["dokumentstatus"]["dokument"]["datum"]
    html = input_dict["dokumentstatus"]["dokument"]["html"]
    html_tree = clean_html(html)
    year = int(date.split("-")[0])
    protocol_id = f"prot-{session}--{pid}"

    data["protocol_id"] = protocol_id
    data["date"] = date.split(" ")[0]
    data["session"] = session

    # New HTML structure with div[@class='Section1']
    section1 = html_tree.xpath(".//div[@class='Section1']")
    for elements in section1:
        for elem in elements:
            if elem.tag in ["p", "h1", "h2"]:
                elemtext = "".join(elem.itertext())
                linebreak = elemtext.strip() == "" and "\n" in elemtext
                if linebreak:
                    pass
                else:
                    paragraph = elemtext.strip()
                    paragraph = paragraph.replace("\n", " ")
                    paragraph = re.sub("\\s+", " ", paragraph)
                    data["paragraphs"].append(paragraph)

    if len(data["paragraphs"]) == 0:
        tree = html_tree

        # Old data structure 1990-2003
        pres = tree.findall(".//pre")
        if len(pres) > 0:
            for pre in pres:
                if pre.text is not None:
                    tblocks = re.sub("([a-zß-ÿ,])- ?\n ?([a-zß-ÿ])", "\\1\\2", pre.text)
                    tblocks = re.sub("([a-zß-ÿ,]) ?\n ?([a-zß-ÿ])", "\\1 \\2", tblocks)
                    for paragraph in tblocks.split("\n"):
                        paragraph = paragraph.replace("\n", " ")
                        paragraph = paragraph.replace("\n", " ")
                        data["paragraphs"].append(paragraph)

        # Standard HTML structure, roughly 2003-2013
        elif len(tree.xpath("//div[@class='indrag']")) > 0:
            tree = tree.xpath("//body")[0]
            for elem in tree:
                elemtext = "".join(elem.itertext())
                linebreak = elemtext.strip() == "" and "\n" in elemtext
                if elem.tag == "br" or linebreak:
                    pass
                else:
                    paragraph = elemtext.strip()
                    paragraph = paragraph.replace("\n", " ")
                    paragraph = re.sub("\\s+", " ", paragraph)
                    data["paragraphs"].append(paragraph)
    return data


In [3]:
protocol_in_question = protocols[10]
root = etree.parse(protocol_in_question, parser).getroot()

In [4]:
protocol_in_question

'data/1867/prot-1867--ak--0130.xml'

In [13]:

for elem in list(paragraph_iterator(root, output="lxml"))[:7]:
  print(" ".join(elem.itertext()))



          RIKSDAGENS Ar PROTOKOLL
        

          1955 ANDRA KAMMAREN Nr 13
        

          13—15 april
        

          Debatter m. m.
        

          Onsdagen den 13 april Sid.
        

          Familjerådgivning «... 5 Interpellation av herr Ericsson : i
          Näs ang. de minskade perioderna
        


In [18]:
data=[]

for i in range(len(protocols)):
  protocol_in_question = protocols[i]


  root = etree.parse(protocol_in_question, parser).getroot()
  element_str=paragraph_iterator(root, output="str")

  data.append({"protocole": protocol_in_question,"texte": " ".join(element_str)})

df=pd.DataFrame(data)
print(df.head())


                           protocole  \
0  data/1867/prot-1867--ak--0118.xml   
1  data/1867/prot-1867--ak--0119.xml   
2  data/1867/prot-1867--ak--0121.xml   
3  data/1867/prot-1867--ak--0122.xml   
4  data/1867/prot-1867--ak--0123.xml   

                                               texte  
0  Sedan, i kraft af Rikets Regeringsform, lagtim...  
1  24 Den 19 Januari. Lördagen den 19 Januari, KL...  
2  Den 21 Januari. 25 mande af tiden för dessa va...  
3  Den 22 Januari. 41 Tisdagen den 22 Januari. Kl...  
4  Den 23 Januari. - 55 Onsdagen den 23 Januari. ...  


In [17]:
print(df.iloc[0]["texte"])

Den 30 Januari, 139 Onsdagen den 30 Januari. KL. 10 f. m. SE Protokollsutdrag för den 28 i denna månad upplästes och godkändes. g 2. Herr L. Öhlin begärde och erhöll ledighet från riksdagsgöromålen under tre dagar. § 3. I enlighet med Kammarens förut fattade beslut företogs nu med slutna sedlar val af ledamöter för inseende af Riksdagens Kansli; och befunnos vid valsedlarnes öppnande till sådane ledamöter vara utsedde: Herr Sjöberg med . «sms c rr rr 122 röster och Leffler nm sssce ses tr 71 ov § 4. Jemlikt Kammarens den 28 i denna månad fattade beslut företogs val af tre ledamöter för att deltaga i talmans-öfverläggningarne. Detta med slutna sedlar förrättade val utföll sålunda att härtill utsågos: Herr Björck med. s .cscs ss ec er 94 röster, » Friherre Gripenstedt med . . . +. - - 77 oo», och » Nils Larson med . ..ssccc ccs 73 8 > Bordlagda motioner hänvisades: Herr Rudbergs motion, N:o 80 till Lag-Utskottet; Herr Heggströms motioner, N:ris 81 och 82, till Stats-Utskottet; Herr .Gren

In [19]:
df.to_pickle("swerick_data_long.pkl")

In [21]:
from datasets import load_dataset
swerick_dataset = load_dataset("pandas",data_files="swerick_data.pkl")

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
print(swerick_dataset)

DatasetDict({
    train: Dataset({
        features: ['protocole', 'texte'],
        num_rows: 130
    })
})


In [20]:
def train_test_valid(df):
    df_train=pd.DataFrame()
    df_test=pd.DataFrame(
    )
    df_valid=pd.DataFrame()

    for s in df["protocole"].unique():
        subset_df = df[df["protocole"] == s]
        h=hash(s)
        h_adjusted = h % (2**32 - 1)
        np.random.seed(h_adjusted)
        r = np.random.rand()
        if r <= 0.7 :
            df_train = pd.concat([df_train, subset_df], ignore_index=True)
        elif r <=0.85:
            df_test = pd.concat([df_test, subset_df], ignore_index=True)
        else:
            df_valid = pd.concat([df_valid, subset_df], ignore_index=True)
    return df_train,df_test,df_valid




In [21]:
df_train,df_test,df_valid=train_test_valid(df)

In [22]:
print(df_test)

                              protocole  \
0     data/1867/prot-1867--ak--0118.xml   
1     data/1867/prot-1867--ak--0121.xml   
2     data/1867/prot-1867--ak--0123.xml   
3     data/1867/prot-1867--ak--0129.xml   
4     data/1867/prot-1867--ak--0130.xml   
...                                 ...   
2678   data/202122/prot-202122--104.xml   
2679   data/202122/prot-202122--106.xml   
2680   data/202122/prot-202122--133.xml   
2681   data/202122/prot-202122--137.xml   
2682   data/202122/prot-202122--141.xml   

                                                  texte  
0     Sedan, i kraft af Rikets Regeringsform, lagtim...  
1     Den 21 Januari. 25 mande af tiden för dessa va...  
2     Den 23 Januari. - 55 Onsdagen den 23 Januari. ...  
3     120 Den 29 Januari, f. m. På Herr Talmannens f...  
4     Den 30 Januari, 139 Onsdagen den 30 Januari. K...  
...                                                 ...  
2678  § 1 Justering av protokoll Protokollet för den...  
2679  § 1 Meddeland

In [23]:
print(df_valid)

                              protocole  \
0     data/1867/prot-1867--ak--0202.xml   
1     data/1867/prot-1867--ak--0213.xml   
2     data/1867/prot-1867--ak--0216.xml   
3     data/1867/prot-1867--ak--0316.xml   
4     data/1867/prot-1867--ak--0318.xml   
...                                 ...   
2635   data/202122/prot-202122--103.xml   
2636   data/202122/prot-202122--117.xml   
2637   data/202122/prot-202122--120.xml   
2638   data/202122/prot-202122--130.xml   
2639   data/202122/prot-202122--138.xml   

                                                  texte  
0     210 Den 2 Februari. Lördagen den 2 Februari. K...  
1     230 Den 13 Februari. Onsdagen den 13 Februari,...  
2     Den 16 Februari, f. m. 239 Lördagen den 16 Feb...  
3     236 Den 16 Mars, f. m. > Lördagen den 16 Mars....  
4     Den 18 Mars, 2938 Bevillnings-Utskottets Betän...  
...                                                 ...  
2635  § 1 Justering av protokoll Protokollet för den...  
2636  § 1 Justering

In [24]:
print(df_train)



                               protocole  \
0      data/1867/prot-1867--ak--0119.xml   
1      data/1867/prot-1867--ak--0122.xml   
2      data/1867/prot-1867--ak--0124.xml   
3      data/1867/prot-1867--ak--0125.xml   
4      data/1867/prot-1867--ak--0126.xml   
...                                  ...   
12314   data/202122/prot-202122--135.xml   
12315   data/202122/prot-202122--136.xml   
12316   data/202122/prot-202122--139.xml   
12317   data/202122/prot-202122--140.xml   
12318   data/202122/prot-202122--142.xml   

                                                   texte  
0      24 Den 19 Januari. Lördagen den 19 Januari, KL...  
1      Den 22 Januari. 41 Tisdagen den 22 Januari. Kl...  
2      60 Den 24 Januari, f. m, Thorsdagen den 24 Jan...  
3      66 Den 25 Januari. N:o 12, med delgifvande af ...  
4      68 Den 26 Januari. Lördagen den 26 Januari. Kl...  
...                                                  ...  
12314  § 1 Justering av protokoll Protokollet för den...  

In [25]:
#df_train,df_test = train_test_split(df,test_size=0.2,random_state=42)
df_train.to_pickle("swerick_data_random_train.pkl")
df_test.to_pickle("swerick_data_random_test.pkl")
df_test.to_pickle("swerick_data_random_valid.pkl")

In [13]:
#datasest
data_files = {"train": "swerick_data_train.pkl", "test": "swerick_data_test.pkl"}
swerick_dataset = load_dataset("pandas",data_files=data_files)
print(swerick_dataset)


DatasetDict({
    train: Dataset({
        features: ['protocole', 'texte', '__index_level_0__'],
        num_rows: 104
    })
    test: Dataset({
        features: ['protocole', 'texte', '__index_level_0__'],
        num_rows: 26
    })
})


In [5]:
protocol_in_question

'data/1867/prot-1867--ak--0202.xml'

In [22]:



notes=[]
intros=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("note")

    for note in note_elements:
        note_text=note.text.strip()
        notes.append(note_text)

        next_element=note.get("type")
        if next_element == "speaker" :
            intros.append(True)
        else :
            intros.append(False)
 
df=pd.DataFrame({"Note" :notes, "Intro":intros})

print(df)

                                                      Note  Intro
0        Sedan, i kraft af Rikets Regeringsform, lagtim...  False
1           Fredagen den 18 Januari 1867, kl. '/, 11 f. m.  False
2        Riksdagens Andra Kammare uti den för dess samm...  False
3                                             Mine Herrar!  False
4        Enligt 33 8 i Riksdags-ordningen skall, innan ...  False
...                                                    ...    ...
7027171  § 7 Bordläggning och beslut om förlängd motion...  False
7027172          § 8 Anmälan om frågor för skriftliga svar  False
7027173           § 9 Anmälan om skriftliga svar på frågor  False
7027174  § 10 Anmälan om uteblivna svar på skriftliga f...  False
7027175                 § 11 Kammaren åtskildes kl. 11.40.  False

[7027176 rows x 2 columns]


In [23]:
df_train_intro,df_test_intro = train_test_split(df,test_size=0.2,random_state=42)
df_train_intro.to_pickle("swerick_data_intro_train.pkl")
df_test_intro.to_pickle("swerick_data_intro_test.pkl")

In [20]:
metadata =load_Corpus_metadata()
print(metadata)


                      person_id       start   end  \
0      i-PFAPNmRqeUAaxDzNRTG1x1        1867  1867   
1      i-QSYHiJ6G54WwZYYDpVnD4u        1867  1867   
2      i-Ddmtm1uG9esPH37c8XjUXZ        1867  1867   
3      i-65rmwEXkkhA1kxSrD4oMUw        1867  1867   
4      i-AvGpsGJvs5PXTcEG4DtbFt        1867  1867   
...                         ...         ...   ...   
13206  i-DhFtpnk9dqFrTXA2tvjJnb  2024-01-30   NaN   
13207  i-9rzEiT5qs2rumTNiqr6kVD  2024-01-31   NaN   
13208  i-Swns2VJiq8mNPgw8cBRfJ9  2024-02-12   NaN   
13209   i-x1CuoKmRHYgQr9i2kh3B5         NaN   NaN   
13210  i-TUyWWYGDFXW92GhiG3CLwF         NaN   NaN   

                                    district                 role  
0          Eskilstuna och Strängnäs valkrets   andrakammarledamot  
1      Västra Götalands läns västra valkrets  förstakammarledamot  
2                      Torna härads valkrets   andrakammarledamot  
3                    Värmlands läns valkrets  förstakammarledamot  
4                Kristi

In [21]:
metadata =pd.DataFrame(metadata)
metadata=metadata.set_index("person_id")
metadata

,start,end,district,role,party,source,chamber,government,born,dead,gender,riksdagen_id,location,name,primary_name,party_abbrev,twitter
person_id,,,,,,,,,,,,,,,,,
i-122QwSSpyGJQiTJjmrUJCM,1992-03-17,1992-05-31,NaN,ledamot,Socialdemokraterna,member_of_parliament,0,NaN,1952-09-22,NaN,man,0573136138218,Västervik,krister örnfjäder,True,S,NaN
i-122QwSSpyGJQiTJjmrUJCM,1992-03-17,1992-05-31,NaN,ledamot,Socialdemokraterna,member_of_parliament,0,NaN,1952-09-22,NaN,man,0573136138218,Västervik,örnfjäder,False,S,NaN
i-122QwSSpyGJQiTJjmrUJCM,1992-11-26,1992-12-31,NaN,ledamot,Socialdemokraterna,member_of_parliament,0,NaN,1952-09-22,NaN,man,0573136138218,Västervik,krister örnfjäder,True,S,NaN
i-122QwSSpyGJQiTJjmrUJCM,1992-11-26,1992-12-31,NaN,ledamot,Socialdemokraterna,member_of_parliament,0,NaN,1952-09-22,NaN,man,0573136138218,Västervik,örnfjäder,False,S,NaN
i-122QwSSpyGJQiTJjmrUJCM,1993-01-11,1993-01-25,NaN,ledamot,Socialdemokraterna,member_of_parliament,0,NaN,1952-09-22,NaN,man,0573136138218,Västervik,krister örnfjäder,True,S,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i-zC3ZtMw7pF7WxkXwbLkKZ,1986-01-10,1988-10-03,NaN,ledamot,Moderaterna,member_of_parliament,0,NaN,1943-10-19,NaN,woman,0233182252702,Åkersberga,sundström,False,M,NaN
i-zC3ZtMw7pF7WxkXwbLkKZ,1988-10-03,1989-01-10,Stockholms läns valkrets,ledamot,Moderaterna,member_of_parliament,0,NaN,1943-10-19,NaN,woman,0233182252702,Åkersberga,gardner,False,M,NaN
i-zC3ZtMw7pF7WxkXwbLkKZ,1988-10-03,1989-01-10,Stockholms läns valkrets,ledamot,Moderaterna,member_of_parliament,0,NaN,1943-10-19,NaN,woman,0233182252702,Åkersberga,ingela gardner sundström,True,M,NaN


In [26]:
type(metadata.loc["i-Ddmtm1uG9esPH37c8XjUXZ" ,"party"].dropna())

pandas.core.series.Series

In [45]:
notes=[]
id=[]
party=[]
gender=[]
protocol=[]

for i in range(len(protocols)):
    protocol_in_question = protocols[i]
    with open(protocol_in_question,"r") as f:
        xml_content=f.read()
    soup=BeautifulSoup(xml_content,"xml")
    note_elements=soup.find_all("u")


    for note in note_elements:
        note_text = note.text.strip()
        note_text = re.sub(r'\s+',' ',note_text)
        note_text = re.sub(r'\n+',' ',note_text)
        next_element = note.get("who")
        if next_element != "unknown":
            id.append(next_element)
            notes.append(note_text)

            party_value = metadata.loc[next_element, "party"]
            if isinstance(party_value, pd.Series):
                party_values = party_value.dropna()
                if not party_values.empty:
                    party.append(party_values.iloc[0])
                else:
                    party.append(np.nan)
            else:
                party.append(party_value)

            gender_value = metadata.loc[next_element, "gender"]
            if isinstance(gender_value, pd.Series):
                gender_values = gender_value.dropna()
                if not gender_values.empty:
                    gender.append(gender_values.iloc[0])
                else:
                    gender.append(np.nan)
            else:
                gender.append(gender_value)

            protocol.append(protocol_in_question)

df = pd.DataFrame({"protocol": protocol, "Note": notes, "id": id, "party": party, "gender": gender})

        

In [44]:
df.iloc[0]["Note"]

'Herr talman! Jag vill inledningsvis påminna om vad som ofta sas inför folkomröstningen om Sverige skulle gå med i EU: EU är ett fredsprojekt. Vi förstod kanske inte vidden av det argumentet då. Men i dag, när ett krig pågår i Europa, när Ryssland krigar mot Ukraina, vet vi hur viktig sammanhållningen i EU är. EU måste stå starkt.'

df

In [29]:
df.to_pickle("gender_party_dataset.pkl")

In [32]:
df = df.rename(columns={"protocol": "protocole"})


In [41]:
df.iloc[0]["Note"]

'Mine Herrar! Sannolikt är det för många bland Eder oväntadt att\n            se en prest intaga talmansstolen i Riksdagens Andra Kammare. Det\n            sker också icke efter min egen önskan. Jag har lika litet eftersträfvat\n            detta höga och ansvarsfulla förtroende, som jag sökt sjelfva riksdagsmannakallets\n            hedrande uppdrag. Men då vahnännen inom den krets jag tillhör\n            lemnat mig det sednare, och Kongl. Maj:t sedermera behagat förläna\n            mig det förra, har jag ej haft giltiga skäl att undandraga mig\n            någotdera. Det är nemligen min öfvertygelse, att hvarje Svensk\n            medborgare, i hvilken samhällsställning han än må befinna sig,\n            är skyldig att gå dit han kallas, så snart kallelsen för honom\n            innebär en uppmaning att, på den plats honom anvisas, efter bästa\n            förstånd bidraga till fäderneslandets gagn. Då I, Mine Herrar,\n            utan tvitvel alla hafven samma öfvertygelse, vågar

In [34]:
df_train,df_test,df_valid=train_test_valid(df)

In [40]:
df_train.iloc[0]["Note"]

'För min del kan jag ej se något hinder för företagande af dessa\n            val ännu något tidigare än Herr Talmannen föreslagit. Jag har\n            föreställt mig, att vi så mycket som möjligt borde taga vara på\n            den dyrbara tiden, och att nu ifrågakomna val kunde förrättas\n            t. ex. på Tisdagen, och valen till suppleanter i Utskotten på\n            Thorsdagen.'

In [35]:
df_train.to_pickle("swerick_data_party_train.pkl")
df_test.to_pickle("swerick_data_party_test.pkl")
df_test.to_pickle("swerick_data_party_valid.pkl")